In [59]:
import pandas as pd
import re
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline

In [60]:
# Wczytanie danych
data = pd.read_csv("data.csv")

In [61]:
# Wyświetlenie ostatnich 10 wierszy
print(data.tail(10))

                                                    Text Language
10327  ಗಾಡಿಯಲ್ಲಿ ಮನೆಯಲ್ಲಿ ನಾರ್ಸಿಸ್ ಅವಳು ಮನೆಗೆ ತಲುಪಿದಾ...  Kannada
10328  ಅವಳು ಮನೆಯಲ್ಲಿ ಕುಳಿತಿದ್ದ ತನ್ನ ತಾಯಿಯನ್ನು ತಬ್ಬಿಕೊ...  Kannada
10329  ಓಹ್ ತಾಯಿ ನಾನು ನಿನ್ನನ್ನು ತುಂಬಾ ಪ್ರೀತಿಸುತ್ತೇನೆ ಮ...  Kannada
10330  ಅವನು ಸುಂದರವಾಗಿದ್ದನು, ಅವನು ನನ್ನ ಬಗ್ಗೆ ಅಷ್ಟಾಗಿ ಕ...  Kannada
10331          ಓಹ್ ಇಲ್ಲ ನೀವು ನೋಡಿದಾಗಿನಿಂದ ಇದು ಸರಿಯಾಗಿದೆ.  Kannada
10332  ನಿಮ್ಮ ತಪ್ಪು ಏನು ಬಂದಿದೆಯೆಂದರೆ ಆ ದಿನದಿಂದ ನಿಮಗೆ ಒ...  Kannada
10333  ನಾರ್ಸಿಸಾ ತಾನು ಮೊದಲಿಗೆ ಹೆಣಗಾಡುತ್ತಿದ್ದ ಮಾರ್ಗಗಳನ್...  Kannada
10334  ಹೇಗೆ ' ನಾರ್ಸಿಸಿಸಮ್ ಈಗ ಮರಿಯನ್ ಅವರಿಗೆ ಸಂಭವಿಸಿದ ಎ...  Kannada
10335  ಅವಳು ಈಗ ಹೆಚ್ಚು ಚಿನ್ನದ ಬ್ರೆಡ್ ಬಯಸುವುದಿಲ್ಲ ಎಂದು ...  Kannada
10336  ಟೆರ್ರಿ ನೀವು ನಿಜವಾಗಿಯೂ ಆ ದೇವದೂತನಂತೆ ಸ್ವಲ್ಪ ಕಾಣು...  Kannada


In [62]:
# Usunięcie niepotrzebnych języków
data = data[~data["Language"].isin(["Arabic", "Danish", "Dutch", "Hindi", "Kannada", "Malayalam", "Sweedish", "Tamil", "Turkish"])]

In [63]:
# Wyświetlenie ostatnich 10 wierszy
print(data.tail(10))

                                                   Text Language
9958  Oh Mama, ich liebe dich so sehr und du hattest...   German
9959  Er war gutaussehend, er hat sich überhaupt nic...   German
9960  Ich weiß, dass es mir so leid tut, wie ich mic...   German
9961  Oh nein, es ist in Ordnung, seit du es gesehen...   German
9962  Was dein Fehler war, lass uns dir von diesem T...   German
9963  narcisa änderte ihre art und weise sie kämpfte...   German
9964  Wie' s Narzissmus jetzt erzählt Marian beiden ...   German
9965  Hat sie, ich denke, sie würde jetzt kein Goldb...   German
9966  Terry, du siehst tatsächlich ein bisschen wie ...   German
9967                              Ihr zwei seid so nett   German


In [64]:
# Podział danych na funkcje (x) i etykiety (y)
x = data["Text"]
y = data["Language"]

In [65]:
# Kodowanie etykiet
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
print("Encoded classes:", label_encoder.classes_)

Encoded classes: ['English' 'French' 'German' 'Greek' 'Italian' 'Portugeese' 'Russian'
 'Spanish']


In [66]:
# Przetwarzanie tekstu
data_list = []
for text in x:
    text = re.sub(r'[!@#$(),\n"%^*?\:;~`0-9\[\]]', ' ', text)
    text = text.lower()
    data_list.append(text)

In [67]:
# Podzieł danych na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(data_list, y_encoded, test_size=0.25, random_state=42)

In [68]:
# Stworzenie modelu Bayesa z wielomianowym modelem naiwnym
vectorizer = CountVectorizer()
model = MultinomialNB()

In [69]:
# Stworzenie potoku do przetwarzania danych
pipe = Pipeline([('vectorizer', vectorizer), ('multinomialNB', model)])

In [70]:
# Dopasowanie modelu do danych treningowych
pipe.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('multinomialNB', MultinomialNB())])

In [71]:
# Predykcja na danych testowych
y_pred = pipe.predict(X_test)

In [72]:
# Ocena dokładności modelu
acc = accuracy_score(y_test, y_pred)
print("Accuracy:", acc)

Accuracy: 0.9877102199223803


In [73]:
# Zapisanie modelu do pliku
with open('ml-project-ue-katowice.pkl', 'wb') as f:
    pickle.dump(pipe, f)

In [74]:
# Kompresja pliku z modelem
import zipfile
with zipfile.ZipFile('ml-project-ue-katowice.pkl.zip', 'w') as zip_file:
    zip_file.write('ml-project-ue-katowice.pkl')